# IPAI
## Individual Challenge - Extracting Knowledge from free form text with an LLM
### Helton Mendonça nº56870

Model taken from https://huggingface.co/AstroMLab/AstroSage-8B
---

As a backup to waht I'm doing in this script I could use https://huggingface.co/spaces/AstroMLab/AstroSage which is the chatbot in a way like ChatGPT, but it's less automated and requires manual pasting of the text which I tried to avoid.
---

From hugginface it says that:

"AstroSage-Llama-3.1-8B is a domain-specialized natural-language AI assistant tailored for research in astronomy, astrophysics, and cosmology. Trained on the complete collection of astronomy-related arXiv papers from 2007-2024 along with millions of synthetically-generated question-answer pairs and other astronomical literature, AstroSage-Llama-3.1-8B demonstrates excellent proficiency on a wide range of questions. "

## Setup

In [ ]:
!pip install bitsandbytes transformers torch accelerate huggingface_hub pandas

In [ ]:
!pip install arxiv

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import arxiv
from arxiv import Client
def collect_abstracts(n_results):
    try:
        client = arxiv.Client()
        # Search for the n most recent articles matching the query
        search = arxiv.Search(
            query="TESS OR JWST OR ALMA OR HAWK-I",
            max_results=n_results,
            sort_by=arxiv.SortCriterion.SubmittedDate
        )
        papers = list(client.results(search))
        if not papers:
            print("No papers found for the given query.")
            return []
        return [paper.summary for paper in papers]
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

For the free text I will be taken arxiv abastracts that mentioned either of our main instruments, with the possibility of exctracting relevant and/or similar knowledge from our original schema of datasets and merging

In [ ]:
def extract_json(response):
    json_start = response.find("{")
    json_end = response.rfind("}")
    if json_start != -1 and json_end != -1:
        json_str = response[json_start:json_end+1]
        try:
            print('Response created Json')
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON decode error: {e}")
            return {}
    else:
        print("Response created empty JSON")
        return {}

# Used ai to help refine this function to make easier to process the json files after

In [ ]:
from tqdm import tqdm

def extract_info(abstracts, prompts):
    results = []
    for k, abstract in enumerate(tqdm(abstracts, total=len(abstracts), desc="Processing abstracts")):
        abstract_data = {}
        for i, prompt in enumerate(tqdm(prompts, total=len(prompts), desc=f"Prompts for abstract {k+1}", leave=False)):
            print(f"Processing prompt {i+1} of {len(prompts)}")
            full_prompt = prompt + f"\nAbstract: {abstract}"
            response = generate_response(full_prompt)
            prompt_data = extract_json(response)
            abstract_data.update(prompt_data)
        results.append(abstract_data)
    return results

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import json
import pandas as pd

# Load the model and tokenizer
quant_config1 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained("AstroMLab/AstroSage-8b",
                                             device_map="auto",
                                             quantization_config=quant_config1)

tokenizer = AutoTokenizer.from_pretrained("AstroMLab/AstroSage-8b")

# Function to generate a response
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    response = outputs[0][inputs['input_ids'].shape[-1]:]
    decoded = tokenizer.decode(response, skip_special_tokens=True)

    return decoded




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/900 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [ ]:
# Example usage
prompt = """
You are an expert in general astrophysics. Your task is to answer the following question:
What are the main components of a galaxy?
"""
response = generate_response(prompt)
print(response)

The main components of a galaxy include stars, gas, dust, and dark matter. Stars are the luminous objects that emit light, gas is the fuel for star formation, dust is the material that can obscure light, and dark matter is the invisible matter that helps to hold the galaxy together. Additionally, galaxies can also contain black holes and other exotic objects. The specific composition of a galaxy can vary depending on its type and size. For example, spiral galaxies like the Milky Way have a central bulge, a disk, and spiral arms, while elliptical galaxies are more concentrated and lack distinct features. Dwarf galaxies, on the other hand


## Prompts and Model Outputs


With a bigger prompt I was getting no results:

(e.g. Consider yourself a astronomy expert, specially in the following instruments:
    TESS, JWST, ALMA, HAWK-I.
    From the following abstract, extract if possible:
    - Target Name
    - Object Type (e.g., star, galaxy, quasar, nebula)
    - Dataset or Observation ID
    - Purpose of Observation
    - Related Observations
    - Key Findings
    - Instrument Used (The telescope, camera, or spectrograph involved)
    - Right Ascencion and Declination (RA and DEC)
    - Observation Date (any format)
    - Exposure Time (any format)
    - Electromagnetic Range (e.g., optical, infrared)
    - Distance from Earth
    - Other possible releventant info I may know) ...



So my strategy will be partioning this bigger prompt into smaller ones combining elements of knowledge i would like to extract from the abastracts. Then i will comnbine into one json per paper to then filter the data and merge it with our original




In [ ]:
prompts = []

prompt1 = f"""Consider yourself a astronomy expert, specially in the following instruments:TESS, JWST, ALMA, HAWK-I.
From the following abstract, extract if exists:
- Target Name
- Object Type (e.g., star, galaxy, quasar, nebula)
- Instrument Used (e.g., TESS, JWST, ALMA, HAWK-I)

Return the result in JSON format. Use null for missing fields.
"""
prompts.append(prompt1)
#####################################
prompt2 = f""" Consider yourself a astronomy expert, specially in the following instruments:TESS, JWST, ALMA, HAWK-I.
From the following abstract, extract if exists:
- Dataset or Observation ID
- Observation Date (any format)

Return the result in JSON format. Use null for missing fields.
"""
prompts.append(prompt2)
#######################################
prompt3 = f""" Consider yourself a astronomy expert, specially in the following instruments:TESS, JWST, ALMA, HAWK-I.
From the following abstract, extract if exists:
- Right Ascension (RA)
- Declination (DEC)
- Distance from Earth

Return the result in JSON format. Use null for missing fields."""
prompts.append(prompt3)
########################################
prompt4 = f""" Consider yourself a astronomy expert, specially in the following instruments:TESS, JWST, ALMA, HAWK-I.
From the following abstract, extract if exists:
- Purpose of Observation
- Key Findings


Return the result in JSON format. Use null for missing fields."""

prompts.append(prompt4)
###################################
prompt5 = f""" Consider yourself a astronomy expert, specially in the following instruments:TESS, JWST, ALMA, HAWK-I.
From the following abstract, extract if exists:
- Exposure Time (any format)
- Electromagnetic Range (e.g., optical, infrared)

Return the result in JSON format. Use null for missing fields."""

prompts.append(prompt5)
##########################################
prompt6 = f""" Consider yourself a astronomy expert, specially in the following instruments:TESS, JWST, ALMA, HAWK-I.
From the following abstract, extract if exists:
- Related Observations
- Any other relevant information

Return the result in JSON format. Use null for missing fields."""

prompts.append(prompt6)

In [ ]:
abstracts = collect_abstracts(n_results=10) # Tried with 100 but went beyond colab gpu limit, so il just use like 20 to prove that it works but ideally for more data diversity the number of papers should be as high as possible

In [ ]:
print(len(abstracts))

20


An example abstract:

In [ ]:
print(abstracts[0])

In this study we incorporate a new grid of kilonova simulations produced by
the Monte Carlo radiative transfer code SuperNu in an inference pipeline for
astrophysical transients, and evaluate their performance. These simulations
contain four different two-component ejecta morphology classes. We analyze
follow-up observational strategies by Vera Rubin Observatory in optical, and
James Webb Space Telescope (JWST) in mid-infrared (MIR). Our analysis suggests
that, within these strategies, it is possible to discriminate between different
morphologies only when late-time JWST observations in MIR are available. We
conclude that follow-ups by the new Vera Rubin Observatory alone are not
sufficient to determine ejecta morphology. Additionally, we make comparisons
between surrogate models based on radiative transfer simulation grids by
SuperNu and POSSIS, by analyzing the historic kilonova AT2017gfo that
accompanied the gravitational wave event GW170817. We show that both SuperNu
and POSSIS mod

In [ ]:
results = extract_info(abstracts=abstracts, prompts=prompts)

Prompts for abstract 1:   0%|          | 0/6 [00:00<?, ?it/s]

Processing prompt 1 of 6



Prompts for abstract 1:  17%|█▋        | 1/6 [00:14<01:12, 14.47s/it]

Response created Json
Processing prompt 2 of 6



Prompts for abstract 1:  33%|███▎      | 2/6 [00:32<01:05, 16.47s/it]

...

Response created Json
JSON decode error: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Processing prompt 5 of 6



Prompts for abstract 10:  83%|████████▎ | 5/6 [01:22<00:16, 16.64s/it]

Response created Json
JSON decode error: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Processing prompt 6 of 6



Processing abstracts: 100%|██████████| 10/10 [14:58<00:00, 89.81s/it]

Response created Json


In [ ]:
results

[{'Target Name': None, 'Object Type': None, 'Instrument Used': None},
 {'Exposure Time': '1 hour', 'Electromagnetic Range': 'Infrared'},
 {'Target Name': None,
  'Object Type': None,
  'Instrument Used': 'JWST',
  'Exposure Time': None,
  'Electromagnetic Range': 'infrared'},
 {'Target Name': None,
  'Object Type': None,
  'Instrument Used': 'TESS',
  'Observation Date': '2023-01-01',
  'Dataset or Observation ID': '10.17909/t9-7s6-5k62'},
 {'Exposure Time': '0.85 - 2.5 μm', 'Electromagnetic Range': 'Infrared'},
 {'Target Name': 'CY Tau',
  'Object Type': 'star',
  'Instrument Used': 'JWST',
  'dataset_or_observation_id': 'JWST-ERS-1324-10601',
  'observation_date': '2023-08-26',
  'Exposure Time': '1.5 hours',
  'Electromagnetic Range': 'infrared'},
 {'Target Name': None,
  'Object Type': None,
  'Instrument Used': ['ALMA', 'SMA', 'MeerKAT'],
  'dataset_or_observation_id': None,
  'observation_date': '2017-10-26'},
 {},
 {},
 {'RelatedObservations': None, 'OtherRelevantInformation': N

## Integration

In [ ]:
import os
from google.colab import files
def save_json_files(results, output_dir="/content/drive/MyDrive/coisas da faculdade/Mestrado Ciência de Dados/2ºsemestre/IPAI/json_files"):
    """Save each JSON object in results as a separate .json file."""
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Loop through results and save each JSON
    for i, result in enumerate(tqdm(results, total=len(results), desc="Saving JSON files")):
        filename = os.path.join(output_dir, f"abstract_{i+1}.json")
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(result, f, indent=4)
        print(f"Saved: {filename}")
        # Download the file in Colab
        files.download(filename)

In [ ]:
save_json_files(results)

Saving JSON files:   0%|          | 0/10 [00:00<?, ?it/s]

Saved: /content/drive/MyDrive/coisas da faculdade/Mestrado Ciência de Dados/2ºsemestre/IPAI/json_files/abstract_1.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving JSON files:  10%|█         | 1/10 [00:00<00:02,  3.30it/s]

...

Saving JSON files:  90%|█████████ | 9/10 [00:00<00:00, 27.18it/s]

Saved: /content/drive/MyDrive/coisas da faculdade/Mestrado Ciência de Dados/2ºsemestre/IPAI/json_files/abstract_10.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving JSON files: 100%|██████████| 10/10 [00:00<00:00, 23.72it/s]


Now after getting the json files, I want to pass them into csv's and merge the equal fields to our "final dataset". Since I couldnt extract much new knowledge I guess its not worth to add to our dataset, but ideally that would be best cause it would enalble us more relevant data, possibly relevant to other questions that we did not explore beforhand

In [5]:
import json
import pandas as pd
import os
current_dataset = pd.read_csv('/content/drive/MyDrive/coisas da faculdade/Mestrado Ciência de Dados/2ºsemestre/IPAI/merged_df.csv')
results = []
for filename in os.listdir('/content/drive/MyDrive/coisas da faculdade/Mestrado Ciência de Dados/2ºsemestre/IPAI/json_files'):
    if filename.endswith('.json'):
        with open(os.path.join('/content/drive/MyDrive/coisas da faculdade/Mestrado Ciência de Dados/2ºsemestre/IPAI/json_files', filename), 'r') as f:
            data = json.load(f)
            print(data)
            df = pd.DataFrame([data])
            results.append(df)



# print(os.getcwd())

{'Target Name': None, 'Object Type': None, 'Instrument Used': None}
{'Exposure Time': '1 hour', 'Electromagnetic Range': 'Infrared'}
{'Target Name': None, 'Object Type': None, 'Instrument Used': 'JWST', 'Exposure Time': None, 'Electromagnetic Range': 'infrared'}
{'Target Name': None, 'Object Type': None, 'Instrument Used': 'TESS', 'Observation Date': '2023-01-01', 'Dataset or Observation ID': '10.17909/t9-7s6-5k62'}
{'Exposure Time': '0.85 - 2.5 μm', 'Electromagnetic Range': 'Infrared'}
{'Target Name': 'CY Tau', 'Object Type': 'star', 'Instrument Used': 'JWST', 'dataset_or_observation_id': 'JWST-ERS-1324-10601', 'observation_date': '2023-08-26', 'Exposure Time': '1.5 hours', 'Electromagnetic Range': 'infrared'}
{'Target Name': None, 'Object Type': None, 'Instrument Used': ['ALMA', 'SMA', 'MeerKAT'], 'dataset_or_observation_id': None, 'observation_date': '2017-10-26'}
{}
{}
{'RelatedObservations': None, 'OtherRelevantInformation': None}


In [10]:
current_dataset.head()

,target_name,s_ra,s_dec,dataset,MJD-OBS,em,coord_block,temp_block,cluster
0,IRAS-05248-7007,81.086467,-70.083778,JWST,59699.184242,3221.5,"(810, -701)",8528,0
1,J1120+0641,170.006167,6.690083,JWST,59945.244998,1147.5,"(1700, 66)",8563,0
2,CEERS-FULL-V2,214.909666,52.872408,JWST,60389.455668,2950.0,"(2149, 528)",8627,0
3,GOODSS2009,53.052791,-27.731936,JWST,60231.322527,2950.0,"(530, -278)",8604,0
4,ABELL2744,3.594322,-30.395694,JWST,60132.288682,3560.0,"(35, -304)",8590,0


In [13]:
results[5]

,Target Name,Object Type,Instrument Used,dataset_or_observation_id,observation_date,Exposure Time,Electromagnetic Range
0,CY Tau,star,JWST,JWST-ERS-1324-10601,2023-08-26,1.5 hours,infrared


In [14]:
results[6]

,Target Name,Object Type,Instrument Used,dataset_or_observation_id,observation_date
0,None,None,"[ALMA, SMA, MeerKAT]",None,2017-10-26


In [15]:
results[4] # As you see the LLM not always get stuff right so we need to take that into account, also shows the importance of infering on more data, and more carefully curated data as well, to avoid these hallucinations

,Exposure Time,Electromagnetic Range
0,0.85 - 2.5 μm,Infrared


Ideally after getting the df's of the json files we would concat them just to get everything and finally merge with our current dataset, in a outer format just to keep all the data fields, which could expand our question pool for example

In [19]:
final_df = pd.concat(results, ignore_index=True)
final_df.head()

,Target Name,Object Type,Instrument Used,Exposure Time,Electromagnetic Range,Observation Date,Dataset or Observation ID,dataset_or_observation_id,observation_date,RelatedObservations,OtherRelevantInformation
0,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,1 hour,Infrared,NaN,NaN,NaN,NaN,NaN,NaN
2,None,None,JWST,None,infrared,NaN,NaN,NaN,NaN,NaN,NaN
3,None,None,TESS,NaN,NaN,2023-01-01,10.17909/t9-7s6-5k62,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,0.85 - 2.5 μm,Infrared,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
final_df = current_dataset.merge(final_df, how='outer', left_on='target_name', right_on='Target Name')
final_df.head()

,target_name_x,s_ra_x,s_dec_x,dataset_x,MJD-OBS_x,em_x,coord_block_x,temp_block_x,cluster_x,target_name_y,...,Object Type,Instrument Used,Exposure Time,Electromagnetic Range,Observation Date,Dataset or Observation ID,dataset_or_observation_id,observation_date,RelatedObservations,OtherRelevantInformation
0,-14-HER,242.602552,43.815628,JWST,60448.923170,1990.5,"(2426, 438)",8635.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-48-Cet-PSF-CALIBRATOR,22.400970,-21.629309,JWST,60302.910535,2950.0,"(224, -217)",8614.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-49-CET,23.658054,-15.676382,JWST,60179.502764,4433.0,"(236, -157)",8597.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-49-CET,23.658063,-15.676382,JWST,60302.799933,2950.0,"(236, -157)",8614.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-49-CET,23.658054,-15.676382,JWST,60169.959282,12800.0,"(236, -157)",8595.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Further Improvements in my opinion

- More specificity in the queries for the papers, meaning a more detailed query to try and find more specific papers on each isntrument or in the problem it self. Some domain knowledge in this area would greatly improve this.

-  Altough I think the queries were clear, more prompt efficiency woulnd't hurt.

- Some cleaning strategies like we did in the first phase by entity matching, field normalization, deduplication, etc.

- And obviously more computational avaibility as colab free verison is quite instable and unpredictable which didnt help my case, but it was all I had since the LLM model was too heavy for my pc.

All of these could refine the use of a LLM in a workflow to produce trustable data on this topic in specific, but Im sure in many other thanks to API's like this arxiv one that enable us to get more trustworthy free text without manual copy pasting.